#  RNN with GloVe Embeddings  
In this notebook, we will look at using trained word embeddings for our headlines data in a RNN model. Since good result for word embedding training can be achieved with big generic corpus, so it is usually more efficient to use existing embedding vectors from other resources as long as the semantics will match with your downstream tasks. After this notebook, you will be able to replace the embedding layer with the GloVe embeddings and tune your RNN model for a sementic analysis task.

**Outline**
- Load and process pretrained GloVe word2vec embeddings
- Build a classification RNN using GloVe embeddings
- Train and evaluate the RNN for semantic analysis  

**Estimated time:** 
 30 mins

In [ ]:
### Change notebook directory, for Gadi environment only
import os
working_path = os.path.expandvars("/scratch/vp91/$USER/Introduction-to-NLP")
os.chdir(working_path)
data_path = '/scratch/vp91/NLP-2024/data/'
model_path = '/scratch/vp91/NLP-2024/model/'

In [ ]:
# local paths
# working_path = './'
# data_path = '../data/'
# model_path = '../model/'

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pickle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")


In [ ]:
# load our tokens back
with open(working_path+'tokens.pkl', 'rb') as f:
    tokens = pickle.load(f)
print(tokens[:10])

## GloVe Embedding


For this workshop, we use a pretrianed GloVe word vectors that is trained from wikipedia in 2014. GloVe models are trained by using word-word co-occurance statistics from the corpus. We are using the smallest word vectors with 6 billion uncased tokens and about 822MB in size.  

Say $x_{ij}$ is the frequency that word $j$ occurs in the context of word $i$, for example, before or after the word $i$ within 10 words. If using $k$ to represent any word, then the frequency for any word to occur in woed $i$ context, we have 

$$x_{i}=\sum_{k} x_{ik}$$  

and the probability of word $j$ to occur in the context of word $i$ (Co-occurance probability) is 
$$p_{ij}=P(w_j\mid w_i)=\frac{x_{ij}}{x_i}$$

Then a function was designed to be equal to $$\frac{p_{ik}}{p_{jk}}$$ so the process of training is to minimise the loss between the designed the function and the fraction of co-occurrance probability.  

[Learn more about GloVe](https://nlp.stanford.edu/projects/glove/)   

<div class="alert alert-block alert-info">

Word vectors is a representation of words co-occurance, so if you want to build a model for ancient literatures, sci-fi novels, hate speeches or other type of text that has very different word-word relationships, you can choose to train your own word vectors by building a word2vec modle in pytorch.
</div>


In [ ]:
# download GoVe
# !curl https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip -o glove.6B.zip
# !unzip glove.6B.zip

Now lets have a look at the word vectors. You can choose different dimensions, 50, 100, 200 or 300 to compare their performance in later tasks. But keep in mind that this will impact the number of dimensions in the neural network as well.

In [ ]:
glove = pd.read_csv(data_path + 'glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove.head()

Now we put it in a dictionary so we can look up a word for its vector.
<div class="alert alert-block alert-warning">
<b>Task 1.</b> <br>
Put glove data in a dictionary, with words being the keys and row vectors being the array values.
</div>

In [ ]:
### TODO
glove_embedding = {}

In [ ]:
glove_embedding['the']

Now we need to extract the vectors from glove to match with our dataset vocabulary. We achieve this by building a weight matrix with the shape of `len(vocabulary) x word_vector_dimension`. This matrix of weights will then be loaded into our embedding layer of the model.

In [ ]:
vocab = list(set([w for t in tokens for w in t]))
vocab_size = len(vocab)
print('Vocabulary size:{}'.format(vocab_size))

<div class="alert alert-block alert-warning">
<b>Task 2. Complete the function: getGloveWeight( )</b><br>
- Input parameter: the vocabulary, glove_embedding dictionary <br><br>
- Output parameter: a weights_matrix of shape vocabulary_length x embedding dimmension <br><br>
If the word in vocabulary exists in glove_embedding, assign the embedding values to the corresbonding row of weights_matrix; otherwise assign a random normal distribution numpy array.

</div>

In [ ]:
def getGloveWeight(vocab, glove_embedding):
    
    ### TODO 2.1: initialise the empty matrix with the correct shape
    weights_matrix =  np.zeros()
    
    for i, w in enumerate(vocab):
        # if word exists in glove embedding, we add the embedding vector to weight matrix
        if w in glove_embedding.keys():
            
            ### TODO 2.2: add the embedding vector to correct place in weight matrix
            weights_matrix[i] = 
        # otherwise we intialise a random variable. 
        else: 
            weights_matrix[i] = np.random.normal(scale=0.6, size=(weights_matrix.shape[1], ))
    return weights_matrix

The embedding layer stores the vectors of all vocabulary in a weight matrix, and accepts indexes for words to extract corresponding rows of vetors for each sequence input. The above function created the weight matrix. So we need to create the word_to_idx and idx_to_word dictionaries.  
To feed the embeddings to the model later, we also need to make all sequenecs of tokens the same length. For padding shorter sequences, we add the special token `<pad>` to our vocabulary.


In [ ]:
glove_embedding['<pad>'] = np.zeros((1,100))
vocab.append('<pad>')

idx_to_word = {}
for i, w in enumerate(vocab):
    idx_to_word[i] = w

word_to_idx = {}
for i, w in enumerate(vocab):
    word_to_idx[w] = i

print(word_to_idx['thirtysomething'])

Now we need to tranform our sequence of tokens to word indexes, also make them all the same length. Here we define sequence length as 15 and pad the short sequences with our special token while truncing the longer sequences.  
  
<div class="alert alert-block alert-warning">
<b>Task 3. Complete the function: formatLength( )</b><br>
- Input parameter: the list of list of tokens, expected length <br><br>
- Output parameter: the result list of list of tokens that all token lists have the expected length <br><br>


</div>


In [ ]:
seq_length = 15
def formatLength(l, seq_length):
    newl=[]
    for i, s in enumerate(l):
        
        ### TODO 3: padding sequences that have length shorter than seq_length
        if len(s)<=seq_length:
            new_s =  
            
        # truncting longer sequences
        else:
            new_s=s[:seq_length]  
        newl.append(new_s)
    return newl

Now we transfer our equal-length token lists from words to index using the word_to_index dictionary. Also confirm that all lists have the same length as specified in seq_length.

In [ ]:
idx_tokens = torch.tensor([[word_to_idx[w] for w in t] for t in formatLength(tokens,seq_length)])
[print(len(l)) for l in idx_tokens if len(l)!=seq_length]

Now let's string above functions together and create the embedding layer with our new weights.
<div class="alert alert-block alert-warning">
<b>Task 4. Complete the function: createEmbeddings( )</b><br>
- Input parameter: the list of vocabulary, glove embedding dictionary <br><br>
- Output parameter: torch embedding layer with GloVe weights. <br><br>

</div>



In [ ]:
# transfer our tokens to vectors
def createEmbeddings(vocab, glove_embedding):
    ### TODO 4.1: create weight matrix from GloVe weights by calling getGloveWeight()
    weights_matrix = getGloveWeight()
    
    # load the weight matrix to a nn.Embedding layer, freeze=True means embedding weights will not be trained 
    ### TODO 4.2: convert weight_matrix to float tensor
    emb_weights =  
    
    ### TODO 4.3: create embedding layer from pretrained weights
    emb_layer = 
    return emb_layer

**Building blocks:**   
- [torch.Tensor](https://pytorch.org/docs/stable/tensors.html#torch.Tensor) 
- [`torch.nn.Embedding` class method `from_pretrained(embeddings, freeze=True, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False`](https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html#Embedding.from_pretrained)

<div class="alert alert-block alert-info"> 
Until now, we introduced static word embeddings, which means the same words will have the same weights in all texts they occur. There is also later developed dynamic word embeddings, which means for each occurence of the same word, the weight vectors depends on the context words around each occurence. You can find out more about <a href="https://dl.acm.org/doi/fullHtml/10.1145/3178876.3185999">dynamic word embeddings</a>.
</div>

# Build RNN

## Prepare Data for Training

In [ ]:
# load our cleaned data back
with open(working_path + 'data_clean.pkl', 'rb') as f:
    data = pickle.load(f)
data.head()

Since we have transferred our data to vectors, we can call `TensorDataset` to load them.

In [ ]:
labels = torch.FloatTensor(data.is_sarcastic.to_list())
# split train test by 80/20
training_data = TensorDataset(idx_tokens[:22895], labels[:22895])
test_data = TensorDataset(idx_tokens[22895:], labels[22895:])
# DataLoader to iterate and batch data quickly
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
training_data[0]

<div class="alert alert-block alert-info"> 
Play with different batch_size and learning rate later to see the model performance.

</div>

## Define simpleRNN  
Here we use the default embeddings first and Train the RNN. You can switch to the Glove Embeddings later and run below cells again to see the performance difference.

In [ ]:
class simpleRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, out_size):
        super(simpleRNN,self).__init__()        
        # TODO use Golve Embeddings, no training on embedding
#         self.embeddings  = createEmbeddings(vocab,glove_embedding) 

        # use original embedding with training
        self.embeddings  = nn.Embedding(vocab_size,embedding_dim)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(embedding_dim, 
                          hidden_size, 
                          num_layers, 
                          batch_first = True,
                          dropout = 0,
                          bidirectional = False
                     )
        self.out = nn.Linear(hidden_size, out_size)
        
    def forward(self, batch):
        batch_size = batch.size()[0]
        embed = self.embeddings(batch)
        rnn_out, hidden = self.rnn(embed)
        result = self.out(rnn_out[:,-1,:])
        return result
    
    

# Training

In [ ]:
embed_dim = len(glove.columns)
hidden_size, num_layers = 50, 1
output_size = 1
vocab_size = len(vocab)

model = simpleRNN(vocab_size, embed_dim, hidden_size, num_layers,output_size)

print(model)

In [ ]:
%%time
print("Number of threads: ", torch.get_num_threads())

# Training
num_epochs, lr = 5, 0.01
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

losses, train_acc= [], []
epochs = []

model.train()
for e in range(num_epochs):
    num_correct = 0
    for d, l in train_dataloader:
        model.zero_grad()
        scores = model(d)
        pred = torch.round(scores.squeeze())
        
        correct_tensor = pred.eq(l.float().view_as(pred))
        correct = np.squeeze(correct_tensor.numpy())
        num_correct += np.sum(correct)
        loss = loss_function(scores.squeeze(), l)
        loss.backward()
        optimizer.step()
    epochs.append(e)
    losses.append(float(loss))
    acc = num_correct/len(train_dataloader.dataset)
    train_acc.append(acc)
    print('Epoch {}, loss: {}, accuracy: {}'.format(e, loss, acc))

# plot the results
plt.title("Training Curve")
plt.plot(losses, label="Train")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

plt.title("Training Curve")
plt.plot(epochs, train_acc, label="Train")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc='best')
plt.show()

# Testing

In [ ]:
# Testing
test_losses = []
num_correct = 0

model.eval()
# iterate over test data
for inputs, labels in test_dataloader:
    # calculate loss
    scores = model(inputs)
    loss = loss_function(scores.squeeze(), labels)    
    test_losses.append(loss.item())
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(scores.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
# avg test loss
print("Test loss: {}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_dataloader.dataset)
print("Test accuracy: {}".format(test_acc))

> Think:  
> What should we do if the model is overfitting?   
>



<div class="alert alert-block alert-warning">
<b>Task 5. Tuning for better performance</b><br>
    1. Change hyperparameters<br>
    2. Change batch_size<br>
    3. Change optimizor<br>
    4. Change nn.RNN parameters
    
    
</div>

**Building blocks:**   
- [`torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False, *, maximize=False, foreach=None)`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)  
- [Other optimizors](https://pytorch.org/docs/stable/optim.html#algorithms)  
- [nn.RNN parameters](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN)

<div class="alert alert-block alert-danger">
<b>Challenge:</b> <br>
    1. Change num_layers to see the result.<br>
    2. Add a dropout layer in the model class to improve the result. <br>
    3. Change model to bidirectional, be careful with the shape match! <br>
    4. Upgrade the model to LSTM or adding Attention layer if you have extra time.
</div>  

------------------------------------------  

Congratulations! After Notebook 1 and 2, you now can clean text data, make tokens, and load the pre-trained weights to an embedding layer in your RNN. In the [next notebook](3-Q&A_bert.ipynb), we will make a small widget to answer questions from a text corpus using fine-tuned transformers. 